In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


1. mon_standard.pkl > array code



In [ ]:
import pickle

USE_SUBLABEL = False
URL_PER_SITE = 10
TOTAL_URLS   = 950

# Load the pickle file
print("Loading datafile...")
with open("/content/drive/MyDrive/기계학습/mon_standard.pkl", 'rb') as fi: # Path to mon_standard.pkl in Colab
    data = pickle.load(fi)

X1 = [] # Array to store instances (timestamps) - 19,000 instances, e.g., [[0.0, 0.5, 3.4, ...], [0.0, 4.5, ...], [0.0, 1.5, ...], ... [... ,45.8]]
X2 = [] # Array to store instances (direction*size) - size information
y = [] # Array to store the site of each instance - 19,000 instances, e.g., [0, 0, 0, 0, 0, 0, ..., 94, 94, 94, 94, 94]

# Differentiate instances and sites, and store them in the respective x and y arrays
# x array (direction*timestamp), y array (site label)
for i in range(TOTAL_URLS):
    if USE_SUBLABEL:
        label = i
    else:
        label = i // URL_PER_SITE # Calculate which site's URL the current URL being processed belongs to and set that value as the label. Thus, URLs fetched from the same site are labeled identically.
    for sample in data[i]:
        size_seq = []
        time_seq = []
        for c in sample:
            dr = 1 if c > 0 else -1
            time_seq.append(abs(c))
            size_seq.append(dr * 512)
        X1.append(time_seq)
        X2.append(size_seq)
        y.append(label)
size = len(y)

print(f'Total samples: {size}') # Output: 19000


Loading datafile...
Total samples: 19000


In [ ]:
# Features
num_total_packets = [] # feature 1
sum_packets = [] # feature 2
num_incoming_packets = [] # feature 3
frac_incoming_packets = [] # feature 4
num_outgoing_packets=[] # feature 5
frac_outgoing_packets=[] # feature 6
average_incoming_ordering=[] # feature 7
std_dev_incoming_ordering=[] # feature 8
average_outgoing_ordering=[] # feature 9
std_dev_outgoing_ordering=[] # feature 10
alternative_packets_per_second_sum = [] # feature 11
mean_of_the_sequence=[] # feature 12

In [ ]:
import numpy as np

for size_seq in X2:
    # 1. Total number of pacekts
    total = len(size_seq)
    num_total_packets.append(total)

    # 2. Sum of packets (absolute value gives the size regardless of direction)
    sum_pckts = sum(abs(size) for size in size_seq)
    sum_packets.append(sum_pckts)

    # 3. Number of incoming packets
    incoming = sum(1 for size in size_seq if size < 0)
    num_incoming_packets.append(incoming)

    # 4. Number of incoming packets as a fraction of the total number of packets
    frac_incoming = incoming / total if total > 0 else 0
    frac_incoming_packets.append(frac_incoming)

    # 5. Number of outgoing packets
    outgoing = sum(1 for size in size_seq if size > 0)
    num_outgoing_packets.append(outgoing)

    # 6. Number of outgoing packets as a fraction of the total number of packets
    frac_outgoing = outgoing / total if total > 0 else 0
    frac_outgoing_packets.append(frac_outgoing)

    # 7. Average of the incoming packet ordering list
    incoming_ordering_list = []
    for idx, size in enumerate(size_seq):
        if size < 0:
            incoming_ordering_list.append(idx)

    average_incoming_ordering.append(np.mean(incoming_ordering_list))

    # 8. Standard deviation of the incoming packet ordering list
    std_dev_incoming_ordering.append(np.std(incoming_ordering_list))

    # 9. Average of the outgoing packet ordering list
    outgoing_ordering_list = []
    for idx, size in enumerate(size_seq):
        if size > 0:
            outgoing_ordering_list.append(idx)
    average_outgoing_ordering.append(np.mean(outgoing_ordering_list))

    # 10. Standard deviation of the outgoing packet ordering list
    std_dev_outgoing_ordering.append(np.std(outgoing_ordering_list))

    # 11. mean of the sequence
    chunks = [size_seq[i:i + 20] for i in range(0, len(size_seq), 20)]
    outgoing_counts = [sum(1 for elem in chunk if elem > 0) for chunk in chunks]
    filtered_array = [element for element in outgoing_counts if element != 0]
    average_outgoing_counts = sum(filtered_array) / len(filtered_array)
    mean_of_the_sequence.append(average_outgoing_counts)

In [ ]:
# 12. Sum of alternative number packets per second
alternative_packets_per_second_sum = []

for time_seq, size_seq in zip(X1, X2):
    packets_per_subset = len(size_seq) // 20 # 20 even-sized subsets
    subset_sums = []

    for i in range(0, len(size_seq), packets_per_subset):
        start_index = i
        end_index = min(i + packets_per_subset, len(size_seq))

        time_interval = time_seq[end_index - 1] - time_seq[start_index]
        time_interval = max(time_interval, 0.001) # Ensure that the time interval does not become zero.

        packets_per_second = (end_index - start_index) / time_interval
        subset_sums.append(packets_per_second)

        if (end_index == packets_per_subset*20):
            break

    alternative_packets_per_second_sum.append(sum(subset_sums))

In [ ]:
print(f"feature 1: {num_total_packets}")
print(f"feature 2: {sum_packets}")
print(f"feature 3: {num_incoming_packets}")
print(f"feature 4: {frac_incoming_packets}")
print(f"feature 5: {num_outgoing_packets}")
print(f"feature 6: {frac_outgoing_packets}")

In [ ]:
print(f'feature 7: {average_outgoing_ordering}');
print(f'feature 8: {std_dev_outgoing_ordering}');
print(f'feature 9: {average_incoming_ordering}');
print(f'feature 10: {std_dev_incoming_ordering}');
print(f'feature 11: {mean_of_the_sequence}');
print(f"feature 12: {alternative_packets_per_second_sum}")

In [ ]:
print(len(num_total_packets))
print(len(sum_packets))
print(len(num_incoming_packets))
print(len(frac_incoming_packets))
print(len(num_outgoing_packets))
print(len(frac_outgoing_packets))
print(len(average_outgoing_ordering))
print(len(std_dev_outgoing_ordering))
print(len(average_incoming_ordering))
print(len(std_dev_incoming_ordering))
print(len(mean_of_the_sequence))
print(len(alternative_packets_per_second_sum))

In [ ]:
incoming_packets_first30 =[] # feature 13
outgoing_packets_first30 = [] # feature 14
transmission_time_Q1 = [] # feature 15
transmission_time_Q2 = [] # feature 16
transmission_time_Q3 = [] # feature 17
transmission_time_Q4 = [] # feature 18

In [ ]:
num_packets_per_second = [] # feature 19
mean_packets_per_second = [] # feature 20
std_packets_per_second = [] # feature 21
med_packets_per_second = [] # feature 22

In [ ]:
# RAM 부족으로 이용X
incoming_max_inter_arrival_times = [] # feature 23
incoming_mean_inter_arrival_times = [] # feature 24
incoming_std_inter_arrival_times = [] # feature 25
incoming_third_quartileinter_arrival_times = [] # feature 26
outgoing_max_inter_arrival_times = [] # feature 27
outgoing_mean_inter_arrival_times = [] # feature 28
outgoing_std_inter_arrival_times = [] # feature 29
outgoing_third_quartileinter_arrival_times = [] # feature 30

In [ ]:
for size_seq in X2:
    incoming_packets = sum(1 for size in size_seq[:30] if size < 0)
    incoming_packets_first30.append(incoming_packets)

    outgoing_packets = sum(1 for size in size_seq[:30] if size > 0)
    outgoing_packets_first30.append(outgoing_packets)

print("incoming_packets_first30:", incoming_packets_first30)
print("outgoing_packets_first30:", outgoing_packets_first30)

In [ ]:
for i in range(len(X1)):
    Q1 = np.percentile(X1[i], 25)
    Q2 = np.percentile(X1[i], 50)
    Q3 = np.percentile(X1[i], 75)
    Q4 = np.percentile(X1[i], 100)

    transmission_time_Q1.append(Q1)
    transmission_time_Q2.append(Q2)
    transmission_time_Q3.append(Q3)
    transmission_time_Q4.append(Q4)

print("transmission_time_Q1:", transmission_time_Q1)
print("transmission_time_Q2:", transmission_time_Q2)
print("transmission_time_Q3:", transmission_time_Q3)
print("transmission_time_Q4:", transmission_time_Q4)

In [ ]:
import statistics

for i in range(len(X1)):
    num_packets = len(X1[i]) / X1[i][-1]
    mean_packets = statistics.mean(X1[i])
    std_packets = statistics.stdev(X1[i])
    med_packets = statistics.median(X1[i])

    num_packets_per_second.append(num_packets)
    mean_packets_per_second.append(mean_packets)
    std_packets_per_second.append(std_packets)
    med_packets_per_second.append(med_packets)

print(f"Num Packets Per Second: {num_packets_per_second}")
print(f"Mean Packets Per Second: {mean_packets_per_second}")
print(f"Std Packets Per Second: {std_packets_per_second}")
print(f"Med Packets Per Second: {med_packets_per_second}")

In [ ]:
import pandas as pd
data = {'num_total_packets': num_total_packets,
        'sum_packets': sum_packets,
        'num_incoming_packets': num_incoming_packets,
        'frac_incoming_packets': frac_incoming_packets,
        'num_outgoing_packets': num_outgoing_packets,
        'frac_outgoing_packets': frac_outgoing_packets,
        'average_outgoing_ordering': average_outgoing_ordering,
        'std_dev_outgoing_ordering': std_dev_outgoing_ordering,
        'average_incoming_ordering': average_incoming_ordering,
        'std_dev_incoming_ordering': std_dev_incoming_ordering,
        'mean_of_the_sequence': mean_of_the_sequence,
        'alternative_packets_per_second_sum': alternative_packets_per_second_sum}
df1 = pd.DataFrame(data)

print(df1.head())

In [ ]:
import pandas as pd
data = {'label': y}
df2 = pd.DataFrame(data)

print(df2.head())

In [ ]:
df1.to_csv('mon_features.csv', index=False)

In [ ]:
df2.to_csv('mon_labels.csv', index=False)

In [ ]:
import pandas as pd
data_modified = {'num_total_packets': num_total_packets, #1
                 'sum_packets': sum_packets, #2
                 'num_incoming_packets': num_incoming_packets, #3
                 'frac_incoming_packets': frac_incoming_packets, #4
                 'num_outgoing_packets': num_outgoing_packets, #5
                 'frac_outgoing_packets': frac_outgoing_packets, #6
                 'average_outgoing_ordering': average_outgoing_ordering, #7
                 'std_dev_outgoing_ordering': std_dev_outgoing_ordering, #8
                 'average_incoming_ordering': average_incoming_ordering, #9
                 'std_dev_incoming_ordering': std_dev_incoming_ordering, #10
                 'mean_of_the_sequence': mean_of_the_sequence, #11
                 'alternative_packets_per_second_sum': alternative_packets_per_second_sum, #12
                 'incoming_packets_first30': incoming_packets_first30, #13
                 'outgoing_packets_first30': outgoing_packets_first30, #14
                 'transmission_time_Q1': transmission_time_Q1, #15
                 'transmission_time_Q2': transmission_time_Q2, #16
                 'transmission_time_Q3': transmission_time_Q3, #17
                 'transmission_time_Q4': transmission_time_Q4, #18
                 'num_packets_per_second': num_packets_per_second, #19
                 'mean_packets_per_second': mean_packets_per_second, #20
                 'std_packets_per_second': std_packets_per_second, #21
                 'med_packets_per_second': med_packets_per_second #22
                 }
df1_modified = pd.DataFrame(data_modified)

print(df1_modified.head())

In [ ]:
df1_modified.to_csv('mon_features_modified1.csv', index=False)

In [ ]:
df1_modified1=pd.read_csv('/content/drive/MyDrive/ML_Project/mon_features_modified1.csv')

In [ ]:
import pandas as pd
data_modified2 = {'incoming_max_inter_arrival_times': incoming_max_inter_arrival_times, #23
                  'incoming_mean_inter_arrival_times': incoming_mean_inter_arrival_times, #24
                  'incoming_std_inter_arrival_times': incoming_std_inter_arrival_times, #25
                  'incoming_third_quartileinter_arrival_times': incoming_third_quartileinter_arrival_times, #26
                  'outgoing_max_inter_arrival_times': outgoing_max_inter_arrival_times, #27
                  'outgoing_mean_inter_arrival_times': outgoing_mean_inter_arrival_times, #28
                  'outgoing_std_inter_arrival_times': outgoing_std_inter_arrival_times, #29
                  'outgoing_third_quartileinter_arrival_times': outgoing_third_quartileinter_arrival_times #30
                 }
df1_modified2 = pd.DataFrame(data_modified2)

print(df1_modified.head())

2. unmon_standard10.pkl > array code

In [ ]:
import pickle

TOTAL_URLS = 10000  # total number in the dataset

# Load 10,000 unmon pickle file
print("Loading datafile...")
with open('/content/drive/MyDrive/ML_Project/unmon_standard.pkl', 'rb') as f:  # Path to unmon_standard10.pkl in Colab
    x = pickle.load(f)

size = len(x)
print(f'Total samples: {size}')

X1 = [] # Array to store instances (timestamps) - 10,000 instances, e.g., [[0.0, 0.5, 3.4, ...], [0.0, 4.5, ...], [0.0, 1.5, ...], ... [... ,45.8]]
X2 = [] # Array to store instances (direction*size) - size information

for i in range(TOTAL_URLS):
    size_seq = []
    time_seq = []
    for c in x[i]:
        dr = 1 if c > 0 else -1
        time_seq.append(abs(c))
        size_seq.append(dr * 512) # In the pickle file, there is no size information, so the conversion code is set to multiply by 512 uniformly.
    X1.append(time_seq)
    X2.append(size_seq)

print(len(X1)) # Print the length of X1

Loading datafile...
Total samples: 10000
10000


In [ ]:
# Features
num_total_packets = [] # feature 1
sum_packets = [] # feature 2
num_incoming_packets = [] # feature 3
frac_incoming_packets = [] # feature 4
num_outgoing_packets=[] # feature 5
frac_outgoing_packets=[] # feature 6
average_incoming_ordering=[] # feature 7
std_dev_incoming_ordering=[] # feature 8
average_outgoing_ordering=[] # feature 9
std_dev_outgoing_ordering=[] # feature 10
alternative_packets_per_second_sum = [] # feature 11
mean_of_the_sequence=[] # feature 12

In [ ]:
import numpy as np

for size_seq in X2:
    # 1. Total number of pacekts
    total = len(size_seq)
    num_total_packets.append(total)

    # 2. Sum of packets (absolute value gives the size regardless of direction)
    sum_pckts = sum(abs(size) for size in size_seq)
    sum_packets.append(sum_pckts)

    # 3. Number of incoming packets
    incoming = sum(1 for size in size_seq if size < 0)
    num_incoming_packets.append(incoming)

    # 4. Number of incoming packets as a fraction of the total number of packets
    frac_incoming = incoming / total if total > 0 else 0
    frac_incoming_packets.append(frac_incoming)

    # 5. Number of outgoing packets
    outgoing = sum(1 for size in size_seq if size > 0)
    num_outgoing_packets.append(outgoing)

    # 6. Number of outgoing packets as a fraction of the total number of packets
    frac_outgoing = outgoing / total if total > 0 else 0
    frac_outgoing_packets.append(frac_outgoing)

    # 7. Average of the incoming packet ordering list
    incoming_ordering_list = []
    for idx, size in enumerate(size_seq):
        if size < 0:
            incoming_ordering_list.append(idx)

    average_incoming_ordering.append(np.mean(incoming_ordering_list))

    # 8. Standard deviation of the incoming packet ordering list
    std_dev_incoming_ordering.append(np.std(incoming_ordering_list))

    # 9. Average of the outgoing packet ordering list
    outgoing_ordering_list = []
    for idx, size in enumerate(size_seq):
        if size > 0:
            outgoing_ordering_list.append(idx)
    average_outgoing_ordering.append(np.mean(outgoing_ordering_list))

    # 10. Standard deviation of the outgoing packet ordering list
    std_dev_outgoing_ordering.append(np.std(outgoing_ordering_list))

    # 11. mean of the sequence
    chunks = [size_seq[i:i + 20] for i in range(0, len(size_seq), 20)]
    outgoing_counts = [sum(1 for elem in chunk if elem > 0) for chunk in chunks]
    filtered_array = [element for element in outgoing_counts if element != 0]
    average_outgoing_counts = sum(filtered_array) / len(filtered_array)
    mean_of_the_sequence.append(average_outgoing_counts)

In [ ]:
# 12. Sum of alternative number packets per second
alternative_packets_per_second_sum = []

for time_seq, size_seq in zip(X1, X2):
    packets_per_subset = len(size_seq) // 20 # 20 even-sized subsets
    subset_sums = []

    for i in range(0, len(size_seq), packets_per_subset):
        start_index = i
        end_index = min(i + packets_per_subset, len(size_seq))

        time_interval = time_seq[end_index - 1] - time_seq[start_index]
        time_interval = max(time_interval, 0.001) # Ensure that the time interval does not become zero.

        packets_per_second = (end_index - start_index) / time_interval
        subset_sums.append(packets_per_second)

        if (end_index == packets_per_subset*20):
            break

    alternative_packets_per_second_sum.append(sum(subset_sums))

In [ ]:
print(f"feature 1: {num_total_packets}")
print(f"feature 2: {sum_packets}")
print(f"feature 3: {num_incoming_packets}")
print(f"feature 4: {frac_incoming_packets}")
print(f"feature 5: {num_outgoing_packets}")
print(f"feature 6: {frac_outgoing_packets}")

In [ ]:
print(f'feature 7: {average_outgoing_ordering}');
print(f'feature 8: {std_dev_outgoing_ordering}');
print(f'feature 9: {average_incoming_ordering}');
print(f'feature 10: {std_dev_incoming_ordering}');
print(f'feature 11: {mean_of_the_sequence}');
print(f"feature 12: {alternative_packets_per_second_sum}")

In [ ]:
incoming_packets_first30 =[] # feature 13
outgoing_packets_first30 = [] # feature 14
transmission_time_Q1 = [] # feature 15
transmission_time_Q2 = [] # feature 16
transmission_time_Q3 = [] # feature 17
transmission_time_Q4 = [] # feature 18

In [ ]:
num_packets_per_second = [] # feature 19
mean_packets_per_second = [] # feature 20
std_packets_per_second = [] # feature 21
min_packets_per_second = [] # feature 22
max_packets_per_second = [] # feature 23
med_packets_per_second = [] # feature 24

In [ ]:
for size_seq in X2:
    incoming_packets = sum(1 for size in size_seq[:30] if size < 0)
    incoming_packets_first30.append(incoming_packets)

    outgoing_packets = sum(1 for size in size_seq[:30] if size > 0)
    outgoing_packets_first30.append(outgoing_packets)

print("incoming_packets_first30:", incoming_packets_first30)
print("outgoing_packets_first30:", outgoing_packets_first30)

In [ ]:
for i in range(len(X1)):
    Q1 = np.percentile(X1[i], 25)
    Q2 = np.percentile(X1[i], 50)
    Q3 = np.percentile(X1[i], 75)
    Q4 = np.percentile(X1[i], 100)

    transmission_time_Q1.append(Q1)
    transmission_time_Q2.append(Q2)
    transmission_time_Q3.append(Q3)
    transmission_time_Q4.append(Q4)

print("transmission_time_Q1:", transmission_time_Q1)
print("transmission_time_Q2:", transmission_time_Q2)
print("transmission_time_Q3:", transmission_time_Q3)
print("transmission_time_Q4:", transmission_time_Q4)

In [ ]:
import statistics

for i in range(len(X1)):
    num_packets = len(X1[i]) / X1[i][-1]
    mean_packets = statistics.mean(X1[i])
    std_packets = statistics.stdev(X1[i])
    med_packets = statistics.median(X1[i])

    num_packets_per_second.append(num_packets)
    mean_packets_per_second.append(mean_packets)
    std_packets_per_second.append(std_packets)
    med_packets_per_second.append(med_packets)

print(f"Num Packets Per Second: {num_packets_per_second}")
print(f"Mean Packets Per Second: {mean_packets_per_second}")
print(f"Std Packets Per Second: {std_packets_per_second}")
print(f"Med Packets Per Second: {med_packets_per_second}")

In [ ]:
import pandas as pd
data = {'num_total_packets': num_total_packets,
        'sum_packets': sum_packets,
        'num_incoming_packets': num_incoming_packets,
        'frac_incoming_packets': frac_incoming_packets,
        'num_outgoing_packets': num_outgoing_packets,
        'frac_outgoing_packets': frac_outgoing_packets,
        'average_outgoing_ordering': average_outgoing_ordering,
        'std_dev_outgoing_ordering': std_dev_outgoing_ordering,
        'average_incoming_ordering': average_incoming_ordering,
        'std_dev_incoming_ordering': std_dev_incoming_ordering,
        'mean_of_the_sequence': mean_of_the_sequence,
        'alternative_packets_per_second_sum': alternative_packets_per_second_sum}
df1 = pd.DataFrame(data)

print(df1.head())

In [ ]:
import pandas as pd

y = [-1] * 10000
data = {'label': y}
df2 = pd.DataFrame(data)

print(df2.head())

In [ ]:
df1.to_csv('unmon_features.csv', index=False)

In [ ]:
df2.to_csv('unmon_labels.csv', index=False)

In [ ]:
import pandas as pd
data_modified = {'num_total_packets': num_total_packets, #1
                 'sum_packets': sum_packets, #2
                 'num_incoming_packets': num_incoming_packets, #3
                 'frac_incoming_packets': frac_incoming_packets, #4
                 'num_outgoing_packets': num_outgoing_packets, #5
                 'frac_outgoing_packets': frac_outgoing_packets, #6
                 'average_outgoing_ordering': average_outgoing_ordering, #7
                 'std_dev_outgoing_ordering': std_dev_outgoing_ordering, #8
                 'average_incoming_ordering': average_incoming_ordering, #9
                 'std_dev_incoming_ordering': std_dev_incoming_ordering, #10
                 'mean_of_the_sequence': mean_of_the_sequence, #11
                 'alternative_packets_per_second_sum': alternative_packets_per_second_sum, #12
                 'incoming_packets_first30': incoming_packets_first30, #13
                 'outgoing_packets_first30': outgoing_packets_first30, #14
                 'transmission_time_Q1': transmission_time_Q1, #15
                 'transmission_time_Q2': transmission_time_Q2, #16
                 'transmission_time_Q3': transmission_time_Q3, #17
                 'transmission_time_Q4': transmission_time_Q4, #18
                 'num_packets_per_second': num_packets_per_second, #19
                 'mean_packets_per_second': mean_packets_per_second, #20
                 'std_packets_per_second': std_packets_per_second, #21
                 'med_packets_per_second': med_packets_per_second #22
                 }
df1_modified = pd.DataFrame(data_modified)

print(df1_modified.head())

In [ ]:
df1_modified.to_csv('unmon_features_modified.csv', index=False)